# Data Preparation - Phase 2 Survey Responses

Note: If there are issues running this script it is likely due to the column names in the raw data - it may need some more cleaning to remove special characters

In [ ]:
import os 
import pandas as pd
from pathlib import Path

from survey_util import load_raw_survey_data, count_by_school_type, calculate_percentage, drop_duplicates

In [ ]:
DATA_DIR = Path('../../data')
SURVEY_DIR = DATA_DIR / 'survey'
RAW_DIR = SURVEY_DIR / 'raw'

VIZ_DIR = Path('../../src/_data/viz/survey/phase_2')

raw_data = load_raw_survey_data()

# Handle duplicates
raw_data_deduped = drop_duplicates(raw_data)

base_respondents = len(raw_data)

SCHOOLS_DATA = pd.read_csv(Path(DATA_DIR, 'leeds_schools_public.csv'))

# dfe_data = pd.read_csv(Path(DATA_DIR, 'spc_ees_school_characteristics.csv'), usecols={
#     'time_period', 
#     'la_name',
#     'school_urn',
#     'school_name',
#     'phase_type_grouping',
#     'typeofestablishment_name',
#     'ward_code',
#     'ward_name',
#     'school_postcode',
#     'school_size'
# })


# Headline stats

In [ ]:
headline_questions = [
    "5_overall_how_would_you_rate_your_schools_arts_curriculum_offer_answers_will_be_anonymised_and_data_will_not_be_linked_to_individual_schools",
    "6_does_your_school_have_a_particular_commitments_to_pupil_entitlement_in_the_arts_eg_every_pupil_learns_a_musical_instrument_or_has_the_opportunity_to_go_on_trips_each_year",
    "7_does_your_school_attract_additional_funding_for_arts_curriculum_or_enrichment_activity",
    "10_does_your_school_actively_sign-post_opportunities_for_pupils_to_develop_creative_skills_and_participate_in_arts_opportunities_beyond_the_curriculum",
    "14_how_would_you_rate_your_schools_extra-curricular_and_arts_enrichment_offer_answers_will_be_anonymised_and_data_will_not_be_linked_to_individual_schools",
    "16_does_your_school_offer_arts_award",
    "18_in_academic_year_202223_did_your_school_invite_organisations_or_creative_practitioners_into_the_school_to_deliver_arts_performances_workshops_or_other_creative_activities",
    "24_does_your_school_have_a_partnership_with_any_local_arts_organisations_ie_a_relationship_with_an_organisation_that_the_school_has_worked_with_over_a_number_of_years"
]

In [ ]:
excellent_arts_curriculum = int(((raw_data["5. Overall, how would you rate your school's arts curriculum offer? (Answers will be anonymised and data will not be linked to individual schools.)"].value_counts().Excellent) / base_respondents * 100))
pupil_arts_entitlement = int((len(raw_data.loc[raw_data['6. Does your school have a particular commitment(s) to pupil entitlement in the arts? (E.g., every pupil learns a musical instrument, or has the opportunity to go on trips each year.)']=='Yes']) / base_respondents * 100))
additional_arts_funding = int((len(raw_data.loc[raw_data['7. Does your school attract additional funding for arts curriculum or enrichment activity?']=='Yes']) / base_respondents * 100))
signposting_beyond_curriculum = int((len(raw_data.loc[raw_data['10. Does your school actively sign-post opportunities for pupils to develop creative skills and participate in arts opportunities beyond the curriculum?']=='Yes']) / base_respondents * 100))
excellent_extra_curricular_arts = int(((raw_data["14. How would you rate your school's extra-curricular and arts enrichment offer? (Answers will be anonymised and data will not be linked to individual schools.)"].value_counts().Excellent) / base_respondents * 100))
arts_award = int((len(raw_data.loc[raw_data['16. Does your school offer Arts Award?']=='Yes']) / base_respondents * 100))
external_orgs_arts_performances = int((len(raw_data.loc[raw_data['18. In academic year 2022/23, did your school invite organisations or creative practitioners into the school to deliver arts performances, workshops, or other creative activities?']=='Yes']) / base_respondents * 100))
partnerships_external_orgs = int((len(raw_data.loc[raw_data['24. Does your school have a partnership with any local arts organisations, i.e. a relationship with an organisation that the school has worked with over a number of years?']=='Yes']) / base_respondents * 100))

headlines_data = {
    'Title': [
        "Arts curriculum",
        "Arts entitlement", 
        "Additional arts funding",
        "Signposting",
        "Extra-curricular offering",
        "Arts award",
        "External practitioners",
        "Local partnerships",
    ],
    'Value': [
        excellent_arts_curriculum,
        pupil_arts_entitlement,
        additional_arts_funding,
        signposting_beyond_curriculum,
        excellent_extra_curricular_arts,
        arts_award,
        external_orgs_arts_performances,
        partnerships_external_orgs
    ],
    'Note': [
        "Rated their arts curriculum as Excellent",
        "Said they have a commitment to pupil entitlement in the arts",
        "Said they seek additional funding for arts curriculum and enrichment",
        "Said they sign-post additional creative opportunities for students",
        "Rated their extra-curricular arts offering as Excellent",
        "Said their school offers an Arts Award",
        "Said they invited external creative practitioners into the school",
        "Said their school has a partnership with local arts organisations"
    ],
    'Suffix': ['%'] * 8
}

headlines = pd.DataFrame(headlines_data)
os.makedirs(VIZ_DIR, exist_ok=True)
headlines.to_csv(os.path.join(VIZ_DIR, 'headlines.csv'), index=False)

## Overview of Respondents

Survey responses by school type

In [ ]:
# Responses broken down by school type

by_school_type = pd.DataFrame({
    'total_respondents' : raw_data_deduped.groupby('03_school_type')['03_school_type'].count(),
    'percent_respondents': raw_data_deduped.groupby('03_school_type')['03_school_type'].count().div(base_respondents).mul(100).round(0).astype(int),
    'unit': '%',
    'notes': [
        'were from a Primary school',
        'were from a Secondary school',
        'were from a Special school',
        'were from a Through school'
    ]
}).to_csv(os.path.join(VIZ_DIR, 'by_school_type.csv'))

## Arts Provision in Schools

Commitment to the arts

In [ ]:
# 06_pupil_arts_entitlement

pupil_arts_entitlement = (
    raw_data_deduped.groupby(['03_school_type', '06_pupil_arts_entitlement'])
    .size()
    .unstack(fill_value=0)
    .apply(lambda x: (x / x.sum()) * 100, axis=1)
    .round(0)
    .astype(int)
    .reset_index()
    .rename(columns={'03_school_type': 'School Type'})
)

pupil_arts_entitlement['Suffix'] = '%'
pupil_arts_entitlement.to_csv(Path(VIZ_DIR, 'pupil_arts_entitlement.csv'), index=False)


In [ ]:
# 05_overall_rating_arts_offer - deduped data
overall_rating_arts_offer = (
    raw_data_deduped.groupby(['03_school_type', '05_overall_rating_arts_offer'])
    .size()
    .unstack(fill_value=0)
    .apply(lambda x: (x / x.sum()) * 100, axis=1)
    .round(0)
    .astype(int)
    .reset_index()
    .rename(columns={'03_school_type': 'School Type'})
    .pivot_table(columns='School Type')
    .reset_index()
    .rename(columns={'index': 'Rating'})
)

overall_rating_arts_offer['Suffix'] = '%'
overall_rating_arts_offer.to_csv(Path(VIZ_DIR, 'overall_rating_arts_offer.csv'), index=False)


In [ ]:
# 14_rating_extra_curricular_arts_enrichment_offering - deduped data

rating_extra_curricular_arts_enrichment_offering = (
    raw_data_deduped.groupby(['03_school_type', '14_rating_extra_curricular_arts_enrichment_offering'])
    .size()
    .unstack(fill_value=0)
    .apply(lambda x: (x / x.sum()) * 100, axis=1)
    .round(0)
    .astype(int)
    .reset_index()
    .rename(columns={'03_school_type': 'School Type'})
    .pivot_table(columns='School Type')
    .reset_index()
    .rename(columns={'index': 'Rating'})
)

rating_extra_curricular_arts_enrichment_offering['Suffix'] = '%'
rating_extra_curricular_arts_enrichment_offering.to_csv(Path(VIZ_DIR, 'rating_extra_curricular_arts_enrichment_offering.csv'), index=False)


Signposting headlines

In [ ]:
# Signposting headlines - percentage of total survey responses

signposting = raw_data.loc[raw_data['10_signposting_beyond_curriculum']=='Yes']
competitions = raw_data.loc[raw_data['09_competitions_showcases']=='Yes']

responses = pd.DataFrame({
    'Competitions & Showcases' : [int(len(competitions)/base_respondents*100)],
    'Signposting beyond the curriculum': [int(len(signposting)/base_respondents*100)]
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'actively sign-post for pupils to develop creative skills and participate in arts opportunities beyond the curriculum',
        'enter pupils into competitions or showcases - whether that is at a local community or regional/national level'
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'signposting_headlines.csv'), index=False)


### Opinions on the arts curriculum

Do you consider your school to lack any particular areas of expertise in delivering the arts curriculum?

Do you consider your school to be an exemplar in any areas of the arts curriculum?

In [ ]:
# 34_school_exemplar_arts_curriculum
# 33_school_lacking_expertise_arts_curriculum

school_exemplar_arts_curriculum = raw_data.loc[raw_data['34_school_exemplar_arts_curriculum']=='Yes']
school_lacking_expertise_arts_curriculum = raw_data.loc[raw_data['33_school_lacking_expertise_arts_curriculum']=='Yes']

responses = pd.DataFrame({
    'Curriculum Exemplar' : [int(len(school_exemplar_arts_curriculum)/base_respondents*100)],
    'Lacking Expertise': [int(len(school_lacking_expertise_arts_curriculum)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'consider their school to be an exemplar in areas of the arts curriculum',
        'consider their school to lack areas of expertise in delivering the arts curriculum',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'school_expertise_headlines.csv'), index=False)


print(school_exemplar_arts_curriculum['34_school_exemplar_arts_curriculum'])

External practitioners

In [ ]:
partnerships_external_orgs = raw_data.loc[raw_data['24_partnerships_external_orgs']=='Yes']
external_orgs_arts_performances = raw_data.loc[raw_data['18_external_orgs_arts_performances']=='Yes']
partnerships_local_schools = raw_data.loc[raw_data['25_partnerships_local_schools']=='Yes']

responses = pd.DataFrame({
    'Partnerships' : [int(len(partnerships_external_orgs)/base_respondents*100)],
    'Delivering activities': [int(len(external_orgs_arts_performances)/base_respondents*100)],
    'School partnerships': [int(len(partnerships_local_schools)/base_respondents*100)]
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'said they have a partnership with any local arts organisations',
        'said their school invites external practitioners to deliver creative activities',
        'said their school partners with other schools to support the arts curriculum'
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'external_partnerships_headlines.csv'), index=False)

Subject areas of the organisations and creative practitioners engaging with schools:

In [ ]:
# Subject areas of external practitioners - deduped data

subject_areas = raw_data_deduped[['02_school_name', '03_school_type', '18a_arts_performance_external_org_subject_areas']].dropna().reset_index()

subject_areas['18a_arts_performance_external_org_subject_areas'] = subject_areas['18a_arts_performance_external_org_subject_areas'].str.split(',')
subject_areas = subject_areas.explode('18a_arts_performance_external_org_subject_areas').drop(columns={'unique_response_number'})

subject_areas = subject_areas.groupby(['03_school_type', '18a_arts_performance_external_org_subject_areas']).count().reset_index()

subject_areas = subject_areas.rename(columns={
    '03_school_type': 'School Type',
    '18a_arts_performance_external_org_subject_areas': 'Subject Area',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

subject_areas['Percent'] = subject_areas.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

subject_areas = subject_areas.pivot_table(columns=['School Type'], index=['Subject Area'], values=['Percent']).reset_index().set_index('Subject Area')

subject_areas.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

subject_areas = subject_areas.fillna(0).astype(int)

subject_areas.to_csv(Path(VIZ_DIR, 'subject_areas_headlines.csv'), index=True)

subject_areas


Community arts programmes:

In [ ]:
# Community programs - total survey responses

community_based_arts_programmes = raw_data.loc[raw_data['26_community_based_arts_programmes']=='Yes']
national_arts_programmes = raw_data.loc[raw_data['23_national_programmes']=='Yes']

responses = pd.DataFrame({
    'Community-based arts programmes' : [int(len(community_based_arts_programmes)/base_respondents*100)],
    'National arts programmes' : [int(len(national_arts_programmes)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'said their school is part of a local/community-based arts programme.',
        'said their school is part of a national arts programme.',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'community_programs_headlines.csv'), index=False)

## Facilities

In [ ]:
# Facilities - deduped data

facilities = raw_data_deduped[['02_school_name', '03_school_type', '13_arts_facilities']].dropna().reset_index()


facilities['13_arts_facilities'] = facilities['13_arts_facilities'].str.split(',')
facilities = facilities.explode('13_arts_facilities').drop(columns={'unique_response_number'})

facilities = facilities.groupby(['03_school_type', '13_arts_facilities']).size().reset_index(name='Count')

counts_by_school_type = count_by_school_type(raw_data_deduped)

facilities['Total'] = facilities.groupby('03_school_type')['Count'].transform('sum').round(0)

facilities['Percent'] = ((facilities['Count'] / facilities['Total']) * 100).round(0).astype(int)

facilities_pivot = facilities.pivot_table(index='13_arts_facilities', columns='03_school_type', values='Percent', fill_value=0)

# facilities_pivot['Total'] = facilities_pivot.sum(axis=1).round(0).astype(int)

facilities_pivot.reset_index(inplace=True)

facilities_pivot.columns.name = None 
facilities_pivot.rename(columns={'Primary': 'Primary', 'Secondary': 'Secondary', 'Special': 'Special School', 'Through': 'Through School'}, inplace=True)

facilities_pivot.to_csv(Path(VIZ_DIR, 'facilities.csv'), index=False)

facilities


## Funding & Support

Does your school attract addition funding for arts curriculum or enrichment activity?

In [ ]:
# 07_additional_arts_funding

funding = raw_data_deduped[['02_school_name', '03_school_type', '07_additional_arts_funding']].dropna().reset_index()

primary_schools = funding.loc[(funding['07_additional_arts_funding'] =='Yes') & (funding['03_school_type'] =='Primary')]
secondary_schools = funding.loc[(funding['07_additional_arts_funding'] =='Yes') & (funding['03_school_type'] =='Secondary')]
through_schools = funding.loc[(funding['07_additional_arts_funding'] =='Yes') & (funding['03_school_type'] =='Through School')]
special_schools = funding.loc[(funding['07_additional_arts_funding'] =='Yes') & (funding['03_school_type'] =='Special School')]

counts_by_school_type = count_by_school_type(raw_data_deduped)

responses = pd.DataFrame({
    'Primary' : [int(len(primary_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Primary'].Count.item())*100)],
    'Secondary' : [int(len(secondary_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Secondary'].Count.item())*100)],
    'Through Schools' : [int(len(through_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Through School'].Count.item())*100)],
    'Special Schools' : [int(len(special_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Special School'].Count.item())*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of Primary schools said they attract additional funding for arts curriculum or enrichment activity.',
        'of Secondary schools said they attract additional funding for arts curriculum or enrichment activity.',
        'of Through schools said they attract additional funding for arts curriculum or enrichment activity.',
        'of Special schools said they attract additional funding for arts curriculum or enrichment activity.',
    ]

funding_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'funding_headlines.csv'), index=False)

Funding sources for provision of art curriculum and enrichment activity:

In [ ]:
# 07a_arts_funding_sources

funding_sources = raw_data_deduped[['02_school_name', '03_school_type', '07a_arts_funding_sources']].dropna().reset_index()

funding_sources['07a_arts_funding_sources'] = funding_sources['07a_arts_funding_sources'].str.split(',')
funding_sources = funding_sources.explode('07a_arts_funding_sources').drop(columns={'unique_response_number'})

funding_sources = funding_sources.groupby(['03_school_type', '07a_arts_funding_sources']).count().reset_index()

funding_sources = funding_sources.rename(columns={
    '03_school_type': 'School Type',
    '07a_arts_funding_sources': 'funding_sources',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

funding_sources['Percent'] = funding_sources.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

funding_sources_pivot = funding_sources.pivot_table(columns=['School Type'], index=['funding_sources'], values=['Percent']).reset_index().set_index('funding_sources')

funding_sources_pivot.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

funding_sources_pivot = funding_sources_pivot.fillna(0).astype(int)

funding_sources_pivot['Suffix'] = '%'

funding_sources_pivot.to_csv(Path(VIZ_DIR, 'funding_sources_headlines.csv'), index=True)

funding_sources_pivot


## Trips

Does your school take pupils on arts trips outside the school?

In [ ]:
# 19_arts_trips

trips = raw_data_deduped[['02_school_name', '03_school_type', '19_arts_trips']].dropna().reset_index()

primary_schools = trips.loc[(trips['19_arts_trips'] =='Yes') & (trips['03_school_type'] =='Primary')]
secondary_schools = trips.loc[(trips['19_arts_trips'] =='Yes') & (trips['03_school_type'] =='Secondary')]
through_schools = trips.loc[(trips['19_arts_trips'] =='Yes') & (trips['03_school_type'] =='Through School')]
special_schools = trips.loc[(trips['19_arts_trips'] =='Yes') & (trips['03_school_type'] =='Special School')]

counts_by_school_type = count_by_school_type(raw_data_deduped)

responses = pd.DataFrame({
    'Primary' : [int(len(primary_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Primary'].Count.item())*100)],
    'Secondary' : [int(len(secondary_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Secondary'].Count.item())*100)],
    'Through Schools' : [int(len(through_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Through School'].Count.item())*100)],
    'Special Schools' : [int(len(special_schools)/(counts_by_school_type[counts_by_school_type['03_school_type']=='Special School'].Count.item())*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of Primary schools said they take pupils on arts trips outside the school.',
        'of Secondary schools said they take pupils on arts trips outside the school.',
        'of Through schools said they take pupils on arts trips outside the school.',
        'of Special schools said they take pupils on arts trips outside the school.',
    ]

arts_trips_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'arts_trips_headlines.csv'), index=False)


Typically, how are your school's arts trips funded?

In [ ]:
# 19a_arts_trips_funding

arts_trips_funding = raw_data_deduped[['02_school_name', '03_school_type', '19a_arts_trips_funding']].dropna().reset_index()

arts_trips_funding['19a_arts_trips_funding'] = arts_trips_funding['19a_arts_trips_funding'].str.split(',')
arts_trips_funding = arts_trips_funding.explode('19a_arts_trips_funding').drop(columns={'unique_response_number'})

arts_trips_funding = arts_trips_funding.groupby(['03_school_type', '19a_arts_trips_funding']).count().reset_index()

arts_trips_funding = arts_trips_funding.rename(columns={
    '03_school_type': 'School Type',
    '19a_arts_trips_funding': 'arts_trips_funding',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

arts_trips_funding['Percent'] = arts_trips_funding.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

arts_trips_funding['arts_trips_funding'] = arts_trips_funding['arts_trips_funding'].str.replace('â€™', "'")
arts_trips_funding['arts_trips_funding'] = arts_trips_funding['arts_trips_funding'].str.replace('From core school funding', "Core school funding")

arts_trips_funding_pivot = arts_trips_funding.pivot_table(columns=['School Type'], index=['arts_trips_funding'], values=['Percent']).reset_index().set_index('arts_trips_funding')

arts_trips_funding_pivot.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

arts_trips_funding_pivot = arts_trips_funding_pivot.fillna(0).astype(int)

arts_trips_funding_pivot['Suffix'] = '%'

arts_trips_funding_pivot.to_csv(Path(VIZ_DIR, 'arts_trips_funding_headlines.csv'), index=True)

arts_trips_funding_pivot

Which of the following did you undertake in academic year 2022/23?

In [ ]:
# 20_trips_undertaken_22_23

trips_undertaken = raw_data_deduped[['02_school_name', '03_school_type', '20_trips_undertaken_22_23']].dropna().reset_index()

trips_undertaken['20_trips_undertaken_22_23'] = trips_undertaken['20_trips_undertaken_22_23'].str.split(',')
trip_types = trips_undertaken.explode('20_trips_undertaken_22_23').drop(columns={'unique_response_number'})

trip_types = trip_types.groupby(['03_school_type', '20_trips_undertaken_22_23']).count().reset_index()

trip_types = trip_types.rename(columns={
    '03_school_type': 'School Type',
    '20_trips_undertaken_22_23': 'trip_types',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

trip_types['Percent'] = trip_types.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

trip_types['trip_types'] = trip_types['trip_types'].str.replace("(", '').str.replace(")", '').str.replace(
    "Trips to a library other than your own school library", 'Trips to a library')

trip_types_pivot = trip_types.pivot_table(columns=['School Type'], index=['trip_types'], values=['Percent']).reset_index().set_index('trip_types')

trip_types_pivot.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

trip_types_pivot = trip_types_pivot.fillna(0).astype(int)

trip_types_pivot['Suffix'] = '%'

trip_types_pivot.to_csv(Path(VIZ_DIR, 'trip_types_headlines.csv'), index=True)


Threatre trip locations...

In [ ]:
# 20b_theatre_trip_locations

theatre_trips = raw_data_deduped[['02_school_name', '03_school_type', '20b_theatre_trip_locations']].dropna().reset_index()

theatre_trips['20b_theatre_trip_locations'] = theatre_trips['20b_theatre_trip_locations'].str.split(',')
theatre_trip_locations = theatre_trips.explode('20b_theatre_trip_locations').drop(columns={'unique_response_number'})

theatre_trip_locations = theatre_trip_locations.groupby(['03_school_type', '20b_theatre_trip_locations']).count().reset_index()

theatre_trip_locations = theatre_trip_locations.rename(columns={
    '03_school_type': 'School Type',
    '20b_theatre_trip_locations': 'theatre_trip_locations',
    '02_school_name': 'Count'
})

counts_by_school_type = count_by_school_type(raw_data_deduped)

theatre_trip_locations['Percent'] = theatre_trip_locations.apply(lambda row: calculate_percentage(row, counts_by_school_type), axis=1)

theatre_trip_locations_pivot = theatre_trip_locations.pivot_table(columns=['School Type'], index=['theatre_trip_locations'], values=['Percent']).reset_index().set_index('theatre_trip_locations')

theatre_trip_locations_pivot.columns = ['Primary', 'Secondary', 'Special School', 'Through School']

theatre_trip_locations_pivot = theatre_trip_locations_pivot.fillna(0).astype(int)

theatre_trip_locations_pivot['Suffix'] = '%'

theatre_trip_locations_pivot.to_csv(Path(VIZ_DIR, 'theatre_trip_locations_headlines.csv'), index=True)

## Barriers

Regarding access, diversity, and inclusion, does your school experience any barriers or challenges in encouraging, recruiting, or gaining permissions for pupils to go on trips out of school?

Regarding access, diversity, and inclusion, does your school experience any barriers or challenges with the experience or facilities at host venues when taking pupils on trips out of the school?

In [ ]:
# 21_barriers_encouraging_recruiting_permissions_trips
# 22_barriers_facilities_host_venues_trips

barriers_encouraging_recruiting_permissions_trips = raw_data.loc[raw_data['21_barriers_encouraging_recruiting_permissions_trips']=='Yes']
barriers_facilities_host_venues_trips = raw_data.loc[raw_data['22_barriers_facilities_host_venues_trips']=='Yes']

responses = pd.DataFrame({
    'Barriers with trips' : [int(len(barriers_encouraging_recruiting_permissions_trips)/base_respondents*100)],
    'Barriers with host venues': [int(len(barriers_facilities_host_venues_trips)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'have experienced barriers in facilitating trips out of school',
        'have experienced barriers or challenges with the experience at host venues',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'barriers_headlines.csv'), index=False)

In [ ]:
# 28_partnerships_diversity_challenges

partnerships_diversity_challenges = raw_data.loc[raw_data['28_partnerships_diversity_challenges']=='Yes']

responses = pd.DataFrame({
    'Diversity challenges' : [int(len(partnerships_diversity_challenges)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'are able to find arts delivery partners who reflect the diversity of their pupils',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'diversity_challenges_headline.csv'), index=False)



## Careers

Careers & student progression headlines

In [ ]:
# 30_specialist_creative_career_advice
# 29_pupil_practitioner_engagement_opportunities
# 32_supporting_teacher_cpd
# 31_data_pupil_progression_creative_education

specialist_creative_career_advice = raw_data.loc[raw_data['30_specialist_creative_career_advice']=='Yes']
pupil_practitioner_engagement_opportunities = raw_data.loc[raw_data['29_pupil_practitioner_engagement_opportunities']=='Yes']
supporting_teacher_cpd = raw_data.loc[raw_data['32_supporting_teacher_cpd']=='Yes']
data_pupil_progression_creative_education = raw_data.loc[raw_data['31_data_pupil_progression_creative_education']=='Yes']


responses = pd.DataFrame({
    'Creative career advice' : [int(len(specialist_creative_career_advice)/base_respondents*100)],
    'Practitioner engagement opportunities': [int(len(pupil_practitioner_engagement_opportunities)/base_respondents*100)],
    'Supporting teacher CPD': [int(len(supporting_teacher_cpd)/base_respondents*100)],
    'Pupil Progression' : [int(len(data_pupil_progression_creative_education)/base_respondents*100)],
}).T.reset_index()

responses.columns = ['Headlines', 'Values']

Note = [
        'of schools have someone able to give specialist advice on career pathways into the arts',
        'give pupils opportunities to engage with arts practitioners',
        "of schools support teachers' CPD in arts subjects",
        'hold data on pupil progression to study arts in further or higher education',
    ]

signposting_headlines = pd.DataFrame({
    'Headlines': responses['Headlines'],
    'Value': responses['Values'],
    'Suffix': '%',
    'Note': Note
}).to_csv(Path(VIZ_DIR, 'careers_headlines.csv'), index=False)

